<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_tachyon_pinn_dropout_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_tachyon_pinn_dropout.py

End‐to‐end script for a physics‐informed, dropout‐based TachyonAI:

  • Synthetic data generator for (E, p, f) → (f_next, rate)
  • PINN loss enforcing df/dt = p − E⋅f
  • Model with residual skip, layer norm, dropout
  • MC Dropout for uncertainty quantification
  • Animated GIF of field evolution

Usage:
  pip install torch matplotlib pillow
  python train_tachyon_pinn_dropout.py
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# ------------------------------------------------------------------------------
# 1. Synthetic Data Generator
# ------------------------------------------------------------------------------
def generate_synthetic_tachyon_data(n_samples=12000, dt=0.1, seed=42):
    np.random.seed(seed)
    E = np.random.uniform(-1, 1, size=(n_samples,1)).astype(np.float32)
    p = np.random.uniform(-1, 1, size=(n_samples,1)).astype(np.float32)
    f = np.random.uniform(-1, 1, size=(n_samples,1)).astype(np.float32)
    X = np.hstack([E, p, f])  # (n_samples,3)

    f_next = f + dt * (p - E*f)
    rate   = p*f - E
    Y = np.hstack([f_next, rate]).astype(np.float32)
    return X, Y

# ------------------------------------------------------------------------------
# 2. Physics‐Informed Neural Net with Dropout & Residual Skip
# ------------------------------------------------------------------------------
class TachyonPINN(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=2, dropout=0.1):
        super().__init__()
        self.fc1      = nn.Linear(input_dim, hidden_dim)
        self.norm1    = nn.LayerNorm(hidden_dim)
        self.relu     = nn.ReLU()
        self.drop     = nn.Dropout(dropout)
        self.fc2      = nn.Linear(hidden_dim, output_dim)
        self.skip     = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        h = self.fc1(x)
        h = self.norm1(h)
        h = self.relu(h)
        h = self.drop(h)
        out = self.fc2(h)
        # residual skip from input
        out = out + self.skip(x)
        return out

# ------------------------------------------------------------------------------
# 3. MC Dropout Prediction for Uncertainty
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, n_samples=50):
    model.train()  # keep dropout on
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            preds.append(model(x).cpu().numpy())
    preds = np.stack(preds, axis=0)  # (n_samples, batch, 2)
    mean = preds.mean(axis=0)
    std  = preds.std(axis=0)
    return mean, std

# ------------------------------------------------------------------------------
# 4. Training & Validation Setup
# ------------------------------------------------------------------------------
def prepare_dataloaders(batch_size=128, val_frac=0.2):
    X, Y = generate_synthetic_tachyon_data()
    X, Y = torch.from_numpy(X), torch.from_numpy(Y)
    ds   = TensorDataset(X, Y)
    n_val= int(len(ds)*val_frac)
    n_trn= len(ds)-n_val
    trn, val = random_split(ds, [n_trn, n_val])
    trn_loader = DataLoader(trn, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)
    return trn_loader, val_loader

# ------------------------------------------------------------------------------
# 5. Loss Function: Data + Physics Constraint
# ------------------------------------------------------------------------------
def pinn_loss(pred, true, inputs, dt=0.1, λ_phys=1.0):
    # supervised MSE
    mse = nn.MSELoss()(pred, true)
    # physics: pred[:,0] ≈ f + dt*(p - E*f)
    f    = inputs[:,2]
    p    = inputs[:,1]
    E    = inputs[:,0]
    f_phys_target = f + dt*(p - E*f)
    f_pred         = pred[:,0]
    phys_loss      = nn.MSELoss()(f_pred, f_phys_target)
    return mse + λ_phys * phys_loss, mse, phys_loss

# ------------------------------------------------------------------------------
# 6. Training Loop with Early Stopping & LR Scheduler
# ------------------------------------------------------------------------------
def train_model(model, trn_loader, val_loader,
                epochs=200, lr=1e-3, weight_decay=1e-5,
                λ_phys=1.0, max_patience=15, device='cpu'):

    optimizer = optim.Adam(model.parameters(), lr=lr,
                           weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5)

    best_val = float('inf')
    patience  = 0
    history = {'train':[], 'val':[]}

    for ep in range(1, epochs+1):
        # Training
        model.train()
        total_loss = total_mse = total_phys = 0
        for xb, yb in trn_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss, mse_l, phys_l = pinn_loss(pred, yb, xb, λ_phys=λ_phys)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*xb.size(0)
            total_mse  += mse_l.item()*xb.size(0)
            total_phys+= phys_l.item()*xb.size(0)

        train_loss = total_loss/len(trn_loader.dataset)
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = pinn_loss(pred, yb, xb, λ_phys=λ_phys)
                val_loss += loss.item()*xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)

        # Early stopping
        if val_loss < best_val:
            best_val = val_loss
            patience  = 0
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            patience += 1
            if patience >= max_patience:
                print(f'Early stopping at epoch {ep}')
                break

        if ep % 10 == 0 or ep==1:
            print(f'Epoch {ep:03d} | Train Loss {train_loss:.4e} | Val Loss {val_loss:.4e}')

    print(f'Best Val Loss: {best_val:.4e}')
    # load best
    model.load_state_dict(torch.load('best_model.pt'))
    return model, history

# ------------------------------------------------------------------------------
# 7. Animate Field Evolution & Save GIF
# ------------------------------------------------------------------------------
def animate_evolution(model, n_steps=50, n_curves=5, dt=0.1,
                      save_path='tachyon_evolution.gif', device='cpu'):
    # sample random initial conditions
    X0 = np.random.uniform(-1,1,(n_curves,3)).astype(np.float32)
    xs = torch.from_numpy(X0).to(device)
    paths = np.zeros((n_curves, n_steps+1), dtype=np.float32)
    paths[:,0] = X0[:,2]

    model.eval()
    with torch.no_grad():
        x = xs
        for t in range(1, n_steps+1):
            pred = model(x)
            f_next = pred[:,0].cpu().numpy()
            paths[:,t] = f_next
            # prepare next input: keep same E,p; update f
            x = torch.stack([x[:,0], x[:,1], torch.from_numpy(f_next).to(device)], dim=1)

    # Plot and animate
    fig, ax = plt.subplots()
    lines = [ax.plot([],[], lw=2)[0] for _ in range(n_curves)]
    ax.set_xlim(0, n_steps*dt)
    ax.set_ylim(paths.min()-0.1, paths.max()+0.1)
    ax.set_xlabel('t')
    ax.set_ylabel('f(t)')
    ax.set_title('Tachyonic Field Evolution')

    def init():
        for line in lines:
            line.set_data([],[])
        return lines

    def update(frame):
        t = np.linspace(0, frame*dt, frame+1)
        for i, line in enumerate(lines):
            line.set_data(t, paths[i,:frame+1])
        return lines

    anim = animation.FuncAnimation(fig, update, frames=n_steps+1,
                                   init_func=init, blit=True)
    anim.save(save_path, writer='pillow', fps=10)
    plt.close(fig)
    print(f'Animation saved to {save_path}')

# ------------------------------------------------------------------------------
# 8. Main Execution
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # data
    train_loader, val_loader = prepare_dataloaders()

    # model
    model = TachyonPINN().to(device)

    # train
    model, history = train_model(model, train_loader, val_loader,
                                 epochs=200, λ_phys=1.0, device=device)

    # plot losses
    plt.figure()
    plt.plot(history['train'], label='Train Loss')
    plt.plot(history['val'],   label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(); plt.title('Training History')
    plt.show()

    # MC Dropout on validation set
    xb, yb = next(iter(val_loader))
    xb = xb.to(device)
    mean, std = mc_dropout_predict(model, xb, n_samples=100)
    print('MC Dropout—first 5 samples:')
    for i in range(5):
        print(f'Input: {xb[i].cpu().numpy()}')
        print(f' Pred Mean: {mean[i]}  Std: {std[i]}')

    # animation
    animate_evolution(model, n_steps=80, n_curves=4, save_path='tachyon_evolution.gif', device=device)